# 6章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

実際のデータを用いてBERTのファインチューニングを行い、その性能を評価するということをやっていく。本章では、livedoorニュースコーパスデータセットを用いて、ニュース記事を9つのカテゴリーに分類する、文章分類のタスクにチャレンジする。

また、本章ではPytorchのラッパーであるPytorch Lightningを用いて実装する。

In [ ]:
# 6-2

!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

文章分類には、`transformsers.BertForSequenceClassification`を用いる。

In [ ]:
# 6-3

import torch
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score

まずはテスト的に2値分類を試みる。<br>
分類モデルには、予め予測するクラス数を`num_labels`引数に指定する。

In [ ]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.cuda() # GPUに載せる

分類する文章とそのラベル(正解データ)を用意する。<br>
`label_list`変数にはカテゴリ値を代入する。二値分類の場合`True`か`False`かの2値となる。

In [ ]:
# 6-5 6-6

text_lst = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]

label_lst = [1, 0, 1] # 0: False, 1: True
labels = torch.tensor(label_lst)

符号化する際に、`labels`変数を辞書型を一緒に代入することができる。

In [ ]:
# 符号化する
encoding = tokenizer(text_lst, padding='longest', return_tensors='pt')
encoding['labels'] = labels

# GPUに載せる
encoding = {key: value.cuda() for key, value in encoding.items()}

`loss`を一緒に出したいので`torch.no_grad()`でブロック化する必要はなし

In [ ]:
# 推論
output = model.forward(**encoding)

# 結果計算
scores = output.logits
labels_pred = scores.argmax(axis=1).cpu()
num_correct = (labels_pred == labels).sum().item()
accuracy = accuracy_score(labels.numpy(), labels_pred.numpy())

# 損失
loss = output.loss

In [ ]:
print(f"labels: {labels}")
print(f"labels pred: {labels_pred}")
print(f"number of correct: {num_correct}")
print(f"accuracy: {accuracy}")
print(f"loss: {loss}")

labels: tensor([1, 0, 1])
labels pred: tensor([0, 0, 0])
number of correct: 1
accuracy: 0.3333333333333333
loss: 0.7982140183448792


In [ ]:
# 6-1
!mkdir chap6
%cd ./chap6

/content/chap6


In [ ]:
# 6-3
import random
import glob
from tqdm import tqdm # プログレスバーの表示

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

In [ ]:
# 6-7
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 

In [ ]:
# 6-8
!cat ./text/it-life-hack/it-life-hack-6342280.txt # ファイルを表示

In [ ]:
# 6-9
# データローダーの作成
dataset_for_loader = [
    {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)},
]
loader = DataLoader(dataset_for_loader, batch_size=2)

# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ## ファインチューニングではここでミニバッチ毎の処理を行う

In [ ]:
# 6-10
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

In [ ]:
# 6-11
# カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す。
        encoding = tokenizer(
            text,
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label # ラベルを追加
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)

In [ ]:
# 6-12
print(dataset_for_loader[0])

In [ ]:
# 6-13
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [ ]:
# 6-14
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
# 6-15
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

In [ ]:
# 6-16
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

In [ ]:
# 6-17
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

In [ ]:
# 6-18
%load_ext tensorboard
%tensorboard --logdir ./

In [ ]:
# 6-19
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
# 6-20
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers') 

In [ ]:
# 6-21
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)